## Hill Climbing

In [1]:
def cost_function(warehouse_locations, customer_locations):
    total_cost = 0
    for customer in customer_locations:
        min_cost = float('inf')
        for warehouse in warehouse_locations:
            distance = abs(customer - warehouse)
            min_cost = min(min_cost, distance)
        total_cost += min_cost
    return total_cost

def generate_neighbors(current_solution, location_range):
    neighbors = []
    for i in range(len(current_solution)):
        if current_solution[i] > location_range[0]:
            neighbor = current_solution.copy()
            neighbor[i] -= 1
            neighbors.append(neighbor)
        if current_solution[i] < location_range[1]:
            neighbor = current_solution.copy()
            neighbor[i] += 1
            neighbors.append(neighbor)
    return neighbors

def hill_climbing(customer_locations, num_warehouses, location_range, max_iter=1000):
    import random
    # Generate initial solution
    current_solution = [random.randint(*location_range) for _ in range(num_warehouses)]
    current_cost = cost_function(current_solution, customer_locations)
    
    for _ in range(max_iter):
        neighbors = generate_neighbors(current_solution, location_range)
        neighbors_costs = [(neighbor, cost_function(neighbor, customer_locations)) for neighbor in neighbors]
        
        # Sort neighbors by cost (ascending)
        neighbors_costs.sort(key=lambda x: x[1])
        
        # Select the best neighbor
        best_neighbor, best_neighbor_cost = neighbors_costs[0]
        
        if best_neighbor_cost < current_cost:
            current_solution = best_neighbor
            current_cost = best_neighbor_cost
        else:
            # No improvement, stop the search
            break
    
    return current_solution, current_cost

# Example usage
customer_locations = [1, 5, 9, 13, 17]
num_warehouses = 2
location_range = (0, 20)

optimal_solution, optimal_cost = hill_climbing(customer_locations, num_warehouses, location_range)
print(f"Optimal Warehouse Locations: {optimal_solution}")
print(f"Total Transportation Cost: {optimal_cost}")


Optimal Warehouse Locations: [13, 4]
Total Transportation Cost: 12


## Simulated Annealing

In [2]:
import random
import math

# Example cost function: total travel distance
def calculate_total_distance(routes, distance_matrix):
    total_distance = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_distance += distance_matrix[route[i]][route[i + 1]]
    return total_distance

# Generate an initial solution: random routes
def generate_initial_solution(customers, num_vehicles):
    random.shuffle(customers)
    routes = [[] for _ in range(num_vehicles)]
    for i, customer in enumerate(customers):
        routes[i % num_vehicles].append(customer)
    return routes

# Generate a neighboring solution by swapping two customers
def generate_neighbor_solution(routes):
    new_routes = [route[:] for route in routes]
    route1, route2 = random.sample(range(len(routes)), 2)
    if not new_routes[route1] or not new_routes[route2]:
        return new_routes
    customer1 = random.choice(new_routes[route1])
    customer2 = random.choice(new_routes[route2])
    new_routes[route1][new_routes[route1].index(customer1)] = customer2
    new_routes[route2][new_routes[route2].index(customer2)] = customer1
    return new_routes

# Simulated Annealing Algorithm
def simulated_annealing(customers, num_vehicles, distance_matrix, initial_temperature, cooling_rate, stopping_temperature):
    current_solution = generate_initial_solution(customers, num_vehicles)
    current_cost = calculate_total_distance(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost
    temperature = initial_temperature
    
    while temperature > stopping_temperature:
        neighbor_solution = generate_neighbor_solution(current_solution)
        neighbor_cost = calculate_total_distance(neighbor_solution, distance_matrix)
        
        if neighbor_cost < current_cost or random.uniform(0, 1) < math.exp((current_cost - neighbor_cost) / temperature):
            current_solution = neighbor_solution
            current_cost = neighbor_cost
            
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost
        
        temperature *= cooling_rate
    
    return best_solution, best_cost

# Example usage
customers = list(range(1, 21))  # Customer locations represented as integers
num_vehicles = 3
distance_matrix = [[random.randint(1, 100) for _ in range(21)] for _ in range(21)]
initial_temperature = 1000
cooling_rate = 0.95
stopping_temperature = 1

optimal_solution, optimal_cost = simulated_annealing(customers, num_vehicles, distance_matrix, initial_temperature, cooling_rate, stopping_temperature)
print(f"Optimal Routes: {optimal_solution}")
print(f"Total Travel Distance: {optimal_cost}")


Optimal Routes: [[15, 11, 6, 8, 3, 13, 10], [9, 20, 17, 12, 18, 1, 4], [19, 14, 5, 2, 16, 7]]
Total Travel Distance: 474
